# Classification of a Signal that Produces Higgs Boson Particles and background signals
# Convolutional Neural Network
### Matthew Boyer and Jonah Goldfine

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc, confusion_matrix
from sklearn.base import BaseEstimator,ClassifierMixin
from sklearn.decomposition import PCA
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from tqdm import tqdm
from skorch import NeuralNetBinaryClassifier
from skorch import callbacks as cb
import optuna
name_dtype=np.array([['class_label', np.float32], ['jet_1_b-tag', np.float64],
            ['jet_1_eta', np.float64], ['jet_1_phi', np.float64],
            ['jet_1_pt', np.float64], ['jet_2_b-tag', np.float64],
            ['jet_2_eta', np.float64], ['jet_2_phi', np.float64],
            ['jet_2_pt', np.float64], ['jet_3_b-tag', np.float64],
            ['jet_3_eta', np.float64], ['jet_3_phi', np.float64],
            ['jet_3_pt', np.float64], ['jet_4_b-tag', np.float64],
            ['jet_4_eta', np.float64], ['jet_4_phi', np.float64],
            ['jet_4_pt', np.float64], ['lepton_eta', np.float64],
            ['lepton_pT', np.float64], ['lepton_phi', np.float64],
            ['m_bb', np.float64], ['m_jj', np.float64],
            ['m_jjj', np.float64], ['m_jlv', np.float64],
            ['m_lv', np.float64], ['m_wbb', np.float64],
            ['m_wwbb', np.float64], ['missing_energy_magnitude', np.float64],
            ['missing_energy_phi', np.float64]])
fullData=pd.read_csv('HIGGS.csv',header=None,names=name_dtype[:,0])
unscaled_X=fullData.drop(['class_label'],axis=1)
scaler=StandardScaler()
full_X=pd.DataFrame(scaler.fit_transform(unscaled_X.values),index=unscaled_X.index,columns=unscaled_X.columns)
full_y=fullData['class_label']
X_train_df,X_test_df,y_train_df,y_test_df=train_test_split(full_X,full_y,test_size=0.2,random_state=0)
pca = PCA(n_components=0.95)  # retaining 95% of the variance
X_train_pca = pca.fit_transform(X_train_df)
X_test_pca = pca.transform(X_test_df)

# Checking the number of components selected and the amount of variance explained
n_components = pca.n_components_
explained_variance = pca.explained_variance_ratio_.sum()

X_train=torch.tensor(X_train_pca).float()
X_test=torch.tensor(X_test_pca).float()
y_train=torch.tensor(y_train_df.values).float()
y_test=torch.tensor(y_test_df.values).float()
class DNN_NoDrop(nn.Module):
    def __init__(self, layer_sizes, activation):
        super(DNN_NoDrop, self).__init__()
        if layer_sizes == 'empty':
            layer_sizes = []
        else:
            layer_sizes = [int(size) for size in layer_sizes.split('_')]

        activation_functions = {'LeakyReLU': nn.LeakyReLU(), 'ReLU': nn.ReLU(), 'Tanh': nn.Tanh()}
        activation = activation_functions[activation]
        self.layers = nn.ModuleList()
        input_size = 23
        for hidden_size in layer_sizes:
            self.layers.append(nn.Linear(input_size, hidden_size))
            self.layers.append(activation)
            input_size = hidden_size

        self.layers.append(nn.Linear(input_size, 1))
        self.layers.append(nn.Sigmoid())

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x.squeeze()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
X_train, y_train = X_train.to(device), y_train.to(device)
dataset = TensorDataset(X_train, y_train)

# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
import time
from torch.cuda.amp import autocast, GradScaler

def objective(trial):
    start_time = time.time()
    layer_sizes = trial.suggest_categorical('layer_sizes', ['empty', '16', '16_8', '64', '64_32', '64_32_16', '64_32_16_8', '128', '128_64', '128_64_32', '128_64_32_16', '128_64_32_16_8'])
    activation = trial.suggest_categorical('activation', ['LeakyReLU', 'ReLU', 'Tanh'])
    max_epochs = trial.suggest_categorical('max_epochs', [10, 25, 50, 100, 250])
    batch_size = 704
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    
    model = DNN_NoDrop(layer_sizes=layer_sizes, activation=activation)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    scaler = GradScaler()
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=16)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=16)
    
    for epoch in range(max_epochs):
        model.train()
        for batch, (input, target) in enumerate(train_loader):
            input, target = input.to(device), target.to(device)
            optimizer.zero_grad()

            with autocast():
                output = model(input)
                loss = criterion(output, target)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad(), autocast():
        for input, target in val_loader:
            input, target = input.to(device), target.to(device)
            output = model(input)
            val_loss += criterion(output, target).item()
            pred = torch.sigmoid(output).ge(0.5).view(-1)
            correct += pred.eq(target.view_as(pred)).sum().item()

    val_loss /= len(val_loader.dataset)
    accuracy = correct / len(val_loader.dataset)
    trial.report(accuracy, epoch)

    # Handle pruning based on the intermediate value
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()
    
    end_time = time.time()
    duration = end_time - start_time
    print(f"Trial {trial.number} took {duration:.2f} seconds.")
    
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

c:\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
best_trial = study.best_trial
best_params = best_trial.params

X_train, y_train = zip(*[data for data in train_dataset])
X_train, y_train = torch.stack(X_train), torch.tensor(y_train)
best_trial.fit(X_train, y_train)

Best trial parameters: {'layer_sizes': '112_56_28_14_7', 'activation': 'LeakyReLU', 'max_epochs': 250, 'batch_size': 200, 'lr': 0.00010022659726287869}
  epoch    accuracy    train_loss    valid_acc    valid_loss      dur
-------  ----------  ------------  -----------  ------------  -------
      1      0.6577        0.6135       0.6895        0.5854  33.3308
      2      0.7018        0.5711       0.7075        0.5629  33.2186
      3      0.7131        0.5548       0.7168        0.5498  33.4217
      4      0.7202        0.5449       0.7216        0.5423  33.1873
      5      0.7244        0.5386       0.7253        0.5371  33.1697
      6      0.7279        0.5335       0.7281        0.5327  33.2076
      7      0.7309        0.5290       0.7307        0.5290  33.4589
      8      0.7335        0.5251       0.7329        0.5257  33.0833
      9      0.7355        0.5218       0.7350        0.5229  33.2814
     10      0.7372        0.5190       0.7361        0.5206  33.3182
     11 

TypeError: save_params got an unexpected argument 'f', did you mean 'f_f'?

In [14]:
# Above error was fixed below, didn't want to re-run code above as it to 20 hours.
import pickle

with open("study.pkl", "wb") as f:
    pickle.dump(study, f)
#best_model.save_params(f='best_model.pth')